<a href="https://colab.research.google.com/github/isaacyeremia/analisis-sentimet-ulasan-review-hotel/blob/main/Deploy_gradio_BiLSTM_dan_DistilBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Deploy BiLSTM dengan Gradio

In [1]:
# --- Import ---
import gradio as gr
import pickle
import re
import nltk
from nltk.corpus import stopwords
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

# --- Setup ---
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# --- Load Model & Tokenizer ---
model = load_model('bilstm_sentiment_model.h5')
with open('tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)

MAX_SEQUENCE_LENGTH = 100

# --- Preprocessing ---
def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    tokens = text.split()
    tokens = [t for t in tokens if t not in stop_words]
    return ' '.join(tokens)

# --- Predict Function ---
def predict_bilstm(text):
    cleaned = preprocess(text)
    seq = tokenizer.texts_to_sequences([cleaned])
    padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)
    prob = model.predict(padded)[0][0]
    label = "Positive 😊" if prob > 0.5 else "Negative 😞"
    return f"{label} ({prob:.4f})"

# --- Gradio App ---
bilstm_interface = gr.Interface(
    fn=predict_bilstm,
    inputs=gr.Textbox(lines=4, placeholder="Masukkan review hotel..."),
    outputs="text",
    title="📊 Hotel Review Sentiment (BiLSTM)",
    description="Masukkan review hotel dan dapatkan prediksi sentimen menggunakan BiLSTM."
)

# --- Jalankan ---
bilstm_interface.launch()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://50b1967658157953ff.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


### Deploy DistilBERT dengan Gradio

In [2]:
# --- Import ---
import gradio as gr
import torch
import re
import nltk
from nltk.corpus import stopwords
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

# --- Setup ---
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# --- Load Model & Tokenizer ---
model_path = "./distilbert_model_v1"
model = DistilBertForSequenceClassification.from_pretrained(model_path).to(device)
tokenizer = DistilBertTokenizer.from_pretrained(model_path)
model.eval()

# --- Preprocessing ---
def preprocess_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.lower()
    return ' '.join([w for w in text.split() if w not in stop_words])

# --- Predict Function ---
def predict_bert(text):
    clean = preprocess_text(text)
    inputs = tokenizer(clean, return_tensors="pt", padding=True, truncation=True, max_length=64)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
        pred = torch.argmax(outputs.logits, dim=1).item()
    label = "Positive 😊" if pred == 1 else "Negative 😞"
    return label

# --- Gradio App ---
bert_interface = gr.Interface(
    fn=predict_bert,
    inputs=gr.Textbox(lines=4, placeholder="Masukkan review hotel..."),
    outputs="text",
    title="🤖 Hotel Review Sentiment (DistilBERT)",
    description="Masukkan review hotel dan dapatkan prediksi sentimen menggunakan DistilBERT."
)

# --- Jalankan ---
bert_interface.launch()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4461d631937507e8b4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
